# Pipeline 2

Produce top 5 entites + gold entities and 0 relations from falcon

In [ ]:
from pipeline import Falcon2Linker, SerialAnnotator, T5Converter
import time
from pprint import pprint

In [ ]:
linker = Falcon2Linker()
annotator = SerialAnnotator()
converter = T5Converter()

In [ ]:
utterance = "Who is the wife of Barack Obama"

In [ ]:
linked = linker.link(utterance)
# rules = [1,2,3,4,5,8,9,10,12,13,14]
# linked = process_text_E_R(utterance, rules)
pprint("Linked")
pprint(linked)

pprint("Annotated")
annotated = annotator.annotate(**linked)
pprint(annotated)

pprint("Converted")
converted = converter.preprocess_inputs(**annotated)
pprint(converted)

In [ ]:
import re

def retrieve_gold_links(masked_wikisparql):
  fragments = masked_wikisparql.upper().strip().split(' ')
  ents = []
  rels = []
  result_dict = {
    'ents': ents,
    'rels': rels,
  }
  for fragment in fragments:
    id_match = re.search('[QP][0-9]+',fragment)
    if id_match:
      id_raw = id_match.group(0)
      if id_raw[0] == 'Q':
        prefix = 'wdt:'
        uri = 'http://www.wikidata.org/entity/' + id_raw
        target_arr = ents
      elif id_raw[0] == 'P':
        prefix = 'wd:'
        uri =  'http://www.wikidata.org/prop/direct/' + id_raw
        target_arr = rels
      else:
        raise ValueError('ID does not start with P or Q')
      match_dict = {
        'id' : id_raw,
        'prefix' : prefix,
        'uri' : uri,
      }
      target_arr.append(match_dict)
  return result_dict

In [ ]:
def pipe(utterance, wikisparql):
    linked = linker.link(utterance)
    annotated = annotator.annotate(**linked)
    converted = converter.preprocess(**annotated, wikisparql=wikisparql)
    return linked, annotated, converted

In [ ]:
def pipe_batch(utterances, wikisparqls):
  batched = []
  link_batch_start = time.time()
  linked = linker.link_batch(utterances)

  assert len(linked) == len(wikisparqls)

  for i, link in enumerate(linked):
    gold_linked = retrieve_gold_links(wikisparqls[i])
    new_ents=link['ents']
    old_ents_set = set(x['id'] for x in link['ents'])
    for gold_ent in gold_linked['ents']:
        if gold_ent['id'] not in old_ents_set:
            new_ents.append(gold_ent)
    link['rels'] = []
    assert len(linked[i]['rels']) == 0

  link_batch_end = time.time()
  s = time.time()
  annotated = annotator.batch_annotate(linked)
  e = time.time()
  assert len(linked) == len(annotated)
  annotator_time = e - s
  converter_time = 0
  for i, single_annotated in enumerate(annotated):
    # annotated = annotator.annotate(**single_linked)
    # try:
    #   annotator_time += e - s
    # except Exception as e:
    #   print("[Annotator Error]:", e)
    #   continue
    s = time.time()
    try:
      converted = converter.preprocess(**single_annotated, wikisparql=wikisparqls[i])
    except Exception as e:
      print("[Converter Error]:", e)
      continue
      
    e = time.time()
    converter_time += e - s
    batched.append([linked[i], annotated[i], converted])
  print("Link batch time:", link_batch_end - link_batch_start)
  print("Anno batch time:", annotator_time)
  print("Conv batch time:", converter_time)
  return batched

## Data

In [ ]:
from pathlib import Path
import pandas as pd
import json

data_path = Path("..") / "t5-for-sparql" / "data" / "lcquad2" / "train.json"
data2_path = Path("..") / "t5-for-sparql" / "data" / "lcquad2" / "test_KG_4211.json"

In [ ]:
with open(data_path) as f:
  data_dict = json.load(f)

with open(data2_path) as f:
  data_dict.extend(json.load(f))

count = 0

for item in data_dict:
  paraphrased_question = item["paraphrased_question"]
  question = item["question"]
  if paraphrased_question == None:
    raise Exception
  if question == None:
    count += 1
    # print(len(paraphrased_question) > 2)
    # pprint(item)
  if item["sparql_wikidata"] == None:
    raise Exception
  

  if len(paraphrased_question) > 2000:
    paraphrased_question = question
  # item["new_question"] = paraphrased_question if len(paraphrased_question) > 2 and paraphrased_question != [] else question
  item["new_question"] = question

print("Missing question count:", count)

df = pd.DataFrame.from_dict(data_dict)
df = df.dropna(axis=0, subset=['new_question'])
df = df[~df.question.isin(['n/a', None, 'none', 'None'])]
n_entries = len(df)
print("Number of rows:", n_entries)

In [ ]:
assert(len(df[df.question.isna()]) == 0)

In [ ]:
for data in df[0:5].iterrows():
    print(data[1].new_question)

In [ ]:
responses = []
batch_qns = []
batch_ans = []
truncated_data = df[:]
total_len = len(truncated_data)
last = 0
cutoff = 0
for i, data in enumerate(truncated_data.iterrows()):
    data = data[1]
    if i < cutoff:
        continue
    # print("Pipeline iter", i)
    question = data["new_question"]
    answer = data["sparql_wikidata"]
    # try
    #     linked, annotated, converted = pipe(question, answer)
    # except Exception as e:
    #     print(e)
    #     continue
    # responses.append([linked, annotated, converted])
    if (question and answer) and len(question) >= 2:
        batch_qns.append(question)
        batch_ans.append(answer)
    if ((i + 1) % 50) == 0 or i == total_len - 1:
        print("[Pipeline1]:", f"Linking {last}-{i}")
        try:
            for linked, annotated, converted in pipe_batch(batch_qns, batch_ans):
                responses.append([linked, annotated, converted])
            batch_qns = []
            batch_ans = []
            last = i
        except Exception as e:
            raise e
            # print(e)
            # continue
        with open(f"../t5-for-sparql/falcon_links/5ents-gold_0rels/link_{i}.json", "w") as f:
            json.dump(responses, f, indent=2, separators=(',',':'))